In [1]:
import requests
from bs4 import BeautifulSoup
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

class ResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for. Example query: Research about the XZ backdoor"
    )


class DuckDuckSearchTool(BaseTool):
    name = "DuckDuckSearchTool"
    description = """
    Use this tool to search for the provided query and provide one url from the searching.
    """
    args_schema: Type[
        ResearchToolArgsSchema
    ] = ResearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class WikiSearchTool(BaseTool):
    name = "WikiSearchTool"
    description = """
    Use this tool to search for the provided query and provide one url from the searching.
    
    """
    args_schema: Type[
        ResearchToolArgsSchema
    ] = ResearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)

class URLSearchToolArgsSchema(BaseModel):
    url: str = Field(
        description="The url you will search. Example url: https://www.techrepublic.com/article/xz-backdoor-linux/",
    )

class LoadWebsiteTool(BaseTool):

    name = "LoadWebsiteTool"
    description = """
    Use this tool to extract one website from either Wikipedia or DuckDuckGo.
    """
    args_schema: Type[URLSearchToolArgsSchema] = URLSearchToolArgsSchema

    def _run(self, url):
        print(url)
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            response.raise_for_status()  # Raises an error for bad responses
            title_text = soup.find('title').text if soup.find('title') else 'Title tag not found'
            return title_text
        except requests.RequestException as e:
            return f"Failed to load URL: {str(e)}"
        except ValueError:
            return "URL did not return a JSON response."
        
class SaveToTextFileToolArgsSchema(BaseModel):
    text: str = Field(
        description="the extracted content from the website will be saved to .txt file.",
    ) 
class SaveToTextFileTool(BaseTool):
    name = "SaveToTextFileTool"
    description = """
    Use this tool to save the text file.
    """
    args_schema: Type[SaveToTextFileToolArgsSchema] = SaveToTextFileToolArgsSchema

    def _run(self, text):
        file_path = f"MyResearch.txt"
        with open(file_path, 'w') as f:
            f.write(text)
        


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
            DuckDuckSearchTool(),
            WikiSearchTool(),
            LoadWebsiteTool(),
            SaveToTextFileTool(),
    ],
)

prompt = "Research about the XZ backdoor"
agent.invoke(prompt)





> Entering new AgentExecutor chain...

Invoking: `DuckDuckSearchTool` with `{'query': 'XZ backdoor'}`


What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A backdoor was planted in the XZ utils package, a popular open-source compression utility for Linux/UNIX systems, by a threat actor named JiaT75. The backdoor exploited the SSH protocol to gain remote access and control over affected devices. The xz-utils package, used by many Linux distributions, had a backdoor that could compromise sshd authentication. Learn how to check and update your Kali installation to avoid this issue. A malicious developer inserted a complex backdoor into the open-source l

{'input': 'Research about the XZ backdoor',
 'output': 'I have gathered information about the XZ backdoor. Here are the key details:\n\n### XZ Utils Backdoor\n\nIn February 2024, a malicious backdoor was introduced to the Linux utility **xz** within the **liblzma** library in versions **5.6.0** and **5.6.1** by an account using the name "Jia Tan". The backdoor allows an attacker who possesses a specific **Ed448** private key to execute remote code on the affected Linux system. This vulnerability has been assigned the Common Vulnerabilities and Exposures (CVE) number **CVE-2024-3094** and has a CVSS score of **10.0**, the highest possible score.\n\nWhile **xz** is commonly included in most Linux distributions, at the time of discovery, the backdoored version had not yet been widely deployed to production systems but was present in development versions of major distributions. The backdoor was discovered by software developer **Andres Freund**, who announced his findings on **29 March 202